In [1]:
import os
import pandas as pd
import gc
import psutil
import warnings
from scipy.spatial import distance_matrix
warnings.filterwarnings('ignore')

In [2]:
def set_df(df):
    df.drop(df[df['Operating_Area'] == '9999999999'].index, inplace=True)
    df = df.dropna(axis=0).reset_index(drop=True)
    df['LAT'] = df['LAT'].astype(float).div(1000000).round(6)
    df['LNG'] = df['LNG'].astype(float).div(1000000).round(6)
    return df

In [3]:
def get_orderby_car_number(df, route, threshold=0.00030):
    bus_stops = route[['LNG', 'LAT']].values
    coords = df[['LNG', 'LAT']].values
    dist_mat = distance_matrix(coords, bus_stops)
    min_dist_array = dist_mat.min(axis=1)
    df['bus_stop'] = min_dist_array <= threshold
    return df[df['bus_stop']][['LNG', 'LAT', 'Information_Occurrence', 'Car_RegistrationNumber']].reset_index(drop=True)

In [4]:
def calculate_chunk_size(file_path, memory_limit):
    sample = pd.read_csv(file_path, sep='|', header=None, nrows=100, encoding='utf-8')
    row_memory = sample.memory_usage(deep=True).sum() / len(sample)
    chunk_size = int(memory_limit / row_memory)
    return chunk_size


In [ ]:
def process_file(file_path, busro, memory_limit, output_folder):
    columns = ['Trip_Key', 'Recorder_Model', 'Car_RentalNumber', 'Car_Type', 'Car_RegistrationNumber',
               'Carrier_RegistrationNumber', 'Driver_Code', 'Day_Drive', 'Total_Drive', 'Car_Speed',
               'Engine_Rotation', 'Break_Signal', 'LNG', 'LAT', 'GIS_Azimuth', 'Acceleration_Vx',
               'Acceleration_Vy', 'Status_Code', 'Operating_Area', 'Information_Occurrence']

    print(f"파일 처리 시작: {file_path}")
    chunk_size = calculate_chunk_size(file_path, memory_limit)

    os.makedirs(output_folder, exist_ok=True)
    output_file_name = os.path.basename(file_path).replace('.gz', '') + '_processed.csv'
    output_path = os.path.join(output_folder, output_file_name)

    is_first_chunk = True
    chunk_num = 1
    for chunk in pd.read_csv(file_path, sep='|', header=None, names=columns, chunksize=chunk_size, encoding='utf-8'):
        print(f"{os.path.basename(file_path)} - 청크 {chunk_num} 처리 중...")
        chunk_num += 1

        chunk = set_df(chunk)
        processed_chunk = get_orderby_car_number(chunk, busro)

        mode = 'w' if is_first_chunk else 'a'
        header_flag = is_first_chunk
        processed_chunk[['LNG', 'LAT', 'Information_Occurrence', 'Car_RegistrationNumber']].to_csv(output_path, index=False, encoding="cp949", mode=mode, header=header_flag)
        is_first_chunk = False

        del chunk, processed_chunk
        gc.collect()

    print(f"파일 처리 및 저장 완료: {output_path}")

In [6]:
def main():
    busro = pd.read_csv("./bus_route_specific.csv", encoding='cp949')
    root = "./data"

    # 처음 스캔 시 파일 리스트 저장
    files = [os.path.join(path, name) for path, _, file_list in os.walk(root) for name in file_list]

    total_memory = psutil.virtual_memory().available
    memory_limit = total_memory * 0.05  # 각 파일 처리 시 메모리 제한

    for file_path in files:
        # 파일 경로에서 상위 폴더 이름 추출
        parent_folder_name = os.path.basename(os.path.dirname(file_path))
        output_folder = f"./{parent_folder_name}_output_map"

        process_file(file_path, busro, memory_limit, output_folder)

    print("모든 파일 처리 완료! 결과는 각 폴더별 *_output_map에 저장되었습니다.")


if __name__ == "__main__":
    main()



파일 처리 시작: ./data\1월_gz\DTG-r-00000.gz
DTG-r-00000.gz - 청크 1 처리 중...
DTG-r-00000.gz - 청크 2 처리 중...
DTG-r-00000.gz - 청크 3 처리 중...
DTG-r-00000.gz - 청크 4 처리 중...
DTG-r-00000.gz - 청크 5 처리 중...
DTG-r-00000.gz - 청크 6 처리 중...
DTG-r-00000.gz - 청크 7 처리 중...
DTG-r-00000.gz - 청크 8 처리 중...
DTG-r-00000.gz - 청크 9 처리 중...
DTG-r-00000.gz - 청크 10 처리 중...
DTG-r-00000.gz - 청크 11 처리 중...
DTG-r-00000.gz - 청크 12 처리 중...


KeyboardInterrupt: 